In [ ]:
from Modele import Modele
from Optimiseur import Optimiseur
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_svmlight_file

In [ ]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# Generate a simple dataset
X, y = make_classification(n_samples=100, n_features=2, n_classes=2, n_informative=2, n_redundant=0, random_state=42)

# Plot the dataset
plt.scatter(X[:, 0], X[:, 1], c=y, cmap='viridis', edgecolor='k')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
plt.show()

In [ ]:
def create_modele(Lambda,gamma,X_train):
    # Create a model and optimizer with the given parameters
    # Initialize the initial vector randomly
    modele = Modele(Lambda,gamma)
    optimiseur = Optimiseur(modele)
    L,mu = modele.constante_L(X_train)
    init= np.random.randn(X_train.shape[0]) 
    return modele, optimiseur, L, mu, init

In [ ]:
# Create the model, optimizer
modele, optimiseur, L, mu, init = create_modele(0,30,X_train)

In [ ]:
def figure(stepsizes,L,train_loss,norm,name1,name2,GD=False):
    """
    Function to plot the stepsizes, train function, and norm of the gradient.
    Parameters:
    - stepsizes: List of stepsizes used in the optimization.
    - L: Lipschitz constant.
    - train_loss: List of training function values through the iterations.
    - norm: List of norms of the gradient values through the iterations.
    - name1: Filename for the stepsizes plot.
    - name2: Filename for the train function and norm plot.
    - GD: Boolean indicating if the optimization is gradient descent.
    """
    if not GD:
        # Plot the stepsizes evolution except if GD
        stepsizes = np.array(stepsizes)
        plt.figure()
        plt.plot(stepsizes * L, label='Stepsizes')
        plt.hlines(y=2, xmin=0, xmax=len(stepsizes), colors='r', linestyles='dashed', label='2')
        plt.xlabel('Iteration', fontsize=14)
        plt.ylabel('Stepsize * L', fontsize=14)
        plt.title('Stepsizes over Iterations', fontsize=16)
        plt.grid()
        plt.savefig(name1)
        plt.show()
        

    # Plot train function on the first subplot
    fig, axs = plt.subplots(1, 2, figsize=(10, 4))
    axs[0].plot(train_loss, label='Train Loss', color='blue')
    axs[0].set_xlabel('Iteration', fontsize=14)
    axs[0].set_ylabel('Train Function', fontsize=14)
    axs[0].set_title('Train Function over Iterations', fontsize=16)
    axs[0].grid()

    # Plot norm on the second subplot
    axs[1].plot(norm, label='Norm', color='orange')
    axs[1].set_xlabel('Iteration', fontsize=14)
    axs[1].set_ylabel('Norm of the Gradient', fontsize=14)
    axs[1].set_title('Norm of the Gradient over Iterations', fontsize=16)
    axs[1].grid()

    plt.tight_layout()
    plt.savefig(name2)
    plt.show()
    

In [ ]:
# Train the model and plot figures using gradient descent with different step sizes

# step size = 1/L
alpha, alpha_list = optimiseur.gradient_descent(X_train, y_train, init.copy(),1/L, 500, 0.001, "norm")
train_loss, test_loss, norm, train_accuracy, test_accuracy = modele.compute_all(X_train,X_test, y_train, y_test, alpha_list)
figure(None,L,train_loss,norm,None,"GD_train_1.pdf",True)

# step size = 2/L
alpha, alpha_list = optimiseur.gradient_descent(X_train, y_train, init.copy(),2/L, 500, 0.001, "norm")
train_loss, test_loss, norm, train_accuracy, test_accuracy = modele.compute_all(X_train,X_test, y_train, y_test, alpha_list)
figure(None,L,train_loss,norm,None,"GD_train_2.pdf",True)

# step size = 2.1/L
alpha, alpha_list = optimiseur.gradient_descent(X_train, y_train, init.copy(),2.1/L, 500, 0.001, "norm")
train_loss, test_loss, norm, train_accuracy, test_accuracy = modele.compute_all(X_train,X_test, y_train, y_test, alpha_list)
figure(None,L,train_loss,norm,None,"GD_train_21.pdf",True)

In [ ]:
# # Train the model and plot figures using exact line serach towards error to optimal step sizes
alpha, alpha_list,stepsizes = optimiseur.optimal_stepsize(X_train, y_train, init.copy(), 1000, 0.001, "norm",True)
train_loss, test_loss, norm, train_accuracy, test_accuracy = modele.compute_all(X_train,X_test, y_train, y_test, alpha_list)
figure(stepsizes,L,train_loss,norm,"optimal_stepsize.pdf","optimal_train.pdf")

In [ ]:
# # Train the model and plot figures using exact line serach towards error vanishing gradient
alpha, alpha_list,stepsizes = optimiseur.exact_stepsize(X_train, y_train, init.copy(), 1000, 0.001, "norm",True)
train_loss, test_loss, norm, train_accuracy, test_accuracy = modele.compute_all(X_train,X_test, y_train, y_test, alpha_list)
figure(stepsizes,L,train_loss,norm,"exact_stepsize.pdf","exact_train.pdf")

In [ ]:
# # Train the model and plot figures using dynamical step size
alpha, alpha_list,stepsizes = optimiseur.dynamic_stepsize(X_train, y_train, init.copy(), 1000, 0.001, "norm",True)
train_loss, test_loss, norm, train_accuracy, test_accuracy = modele.compute_all(X_train,X_test, y_train, y_test, alpha_list)
figure(stepsizes,L,train_loss,norm,"dynamic_stepsize.pdf","dynamic_train.pdf")

In [ ]:
# Train the model and plot figures using periodically large step sizes for different number of iterations

# stop when converging at 0.001
alpha, alpha_list,stepsizes = optimiseur.Periodic(X_train, y_train, init.copy(), 1000, 0.001, "norm",True)
train_loss, test_loss, norm, train_accuracy, test_accuracy = modele.compute_all(X_train,X_test, y_train, y_test, alpha_list)
figure(stepsizes,L,train_loss,norm,"periodic_stepsize.pdf","periodic_train.pdf")

# stop after 127 iterations
alpha, alpha_list,stepsizes = optimiseur.Periodic(X_train, y_train, init.copy(), 127, 0.000001, "norm",True)
train_loss, test_loss, norm, train_accuracy, test_accuracy = modele.compute_all(X_train,X_test, y_train, y_test, alpha_list)
figure(stepsizes,L,train_loss,norm,"periodic_stepsize_127.pdf","periodic_train_127.pdf")

# stop after 500 iterations
alpha, alpha_list,stepsizes = optimiseur.Periodic(X_train, y_train, init.copy(), 500, 0.000001, "norm",True)
train_loss, test_loss, norm, train_accuracy, test_accuracy = modele.compute_all(X_train,X_test, y_train, y_test, alpha_list)
figure(stepsizes,L,train_loss,norm,"periodic_stepsize_500.pdf","periodic_train_500.pdf")